In [1]:
from monitor import utc_to_tz
from dateutil import tz
from datetime import datetime
import krakenex
import pymongo
import networkx as nx
import pandas
import numpy as np
import tqdm
import seaborn
from matplotlib import pyplot as plt
import collections

In [2]:
def kraken_ticker_to_last_prices(ticker):
    kraken_fee = 1-0.26/100.
    prices = {}
    for asset_name in ticker:
        if 'DASH' in asset_name:
            continue
        assert len(asset_name) == 8
        base = asset_name[:4]
        quote = asset_name[4:]
        assert (base, quote) not in prices
        assert (quote, base) not in prices
        last_price, last_volume = map(float, ticker[asset_name]['c'])
        assert last_price > 0
        prices[base, quote] = last_price * kraken_fee
        prices[quote, base] = 1./last_price * kraken_fee
    return prices

In [3]:
def poloniex_ticker_to_last_prices(ticker):
    poloniex_fee = 1-0.25/100.
    prices = {}
    for asset_name in ticker:
        base, quote = asset_name.split('_')
        assert (base, quote) not in prices
        assert (quote, base) not in prices
        last_price = float(ticker[asset_name]['last'])
        assert last_price > 0
        prices[base, quote] = last_price * poloniex_fee
        prices[quote, base] = 1./last_price * poloniex_fee
    return prices

In [ ]:
def xbtce_ticker_to_last_prices(ticker):
    prices = {}
    for asset in ticker:
        name = asset['Symbol']
        assert len(name) == 6
        base = name[:3]
        quote = name[4:]
        assert (base, quote) not in prices
        assert (quote, base) not in prices
        last_buy_price = asset['LastBuyPrice']
        last_sell_price = asset['LastSellPrice']
        prices[base, quote] = last_sell_price
        assert last_sell_price > 0
        prices[quote, base] = last_buy_price
        assert last_buy_price > 0
    return prices